In [1]:
import random
from faker import Faker
import psycopg2
from psycopg2.extras import execute_values

fake = Faker("vi_VN")

## Create customer

In [2]:
customers = []
customer_ids = []

for _ in range(700):
    customer = {
        "customer_name": fake.name(),
        "phone_number": fake.phone_number(),
        "cccd": ''.join(str(random.randint(0, 9)) for _ in range(12))
    }
    customers.append(customer)

print(len(customers))
print(customers[0])

700
{'customer_name': 'Bảo Hải Trần', 'phone_number': '+84-09-994 5601', 'cccd': '335265132316'}


In [3]:
conn = psycopg2.connect(
    host="aws-1-ap-south-1.pooler.supabase.com",
    database="postgres",
    user="postgres.rruavcjmtgpxyznzwkhw",
    password="khaibaolocnguyen",
    port=5432
)
cur = conn.cursor()

query = """
INSERT INTO customer (customer_name, phone_number, cccd)
VALUES %s
RETURNING customer_id
"""

values = [
    (c["customer_name"], c["phone_number"], c["cccd"])
    for c in customers
]

execute_values(cur, query, values, page_size=len(values))
customer_ids = [row[0] for row in cur.fetchall()]
conn.commit()
print(len(customer_ids))

700


## Create account

In [6]:
accounts = []
account_ids = []

used_emails = set()
used_usernames = set()

for i in range(700):
    username = f"customer_{i:04d}"
    email = f"customer_{i:04d}@puppylobby.vn"

    accounts.append({
        "username": username,
        "password": "User@123",
        "email": email,
        "role_id": "afb999bb-b2df-4e21-97b6-66185cdb7f84"
    })
  
print(len(accounts))

700


In [7]:
query = """
INSERT INTO account (username, password, email, role_id)
VALUES %s
RETURNING account_id
"""

values = [
    (a["username"], a["password"], a["email"], a["role_id"])
    for a in accounts
]

execute_values(cur, query, values, page_size=len(values))

account_ids = [row[0] for row in cur.fetchall()]

conn.commit()

## Tạo customer account dựa trên account_ids và customer_ids

In [8]:
customer_accounts = []

for i in range(700):
    customer_account = {
        "account_id": account_ids[i],
        "loyalty_score": 0,
        "reach_target": 0,
        "customer_id": customer_ids[i],
        "level_id": "7c5b193e-ff68-42b2-bfb5-0a9eb7dcfa8f" # default level id
    }
    customer_accounts.append(customer_account)

print(len(customer_accounts))

700


In [9]:
query = """
INSERT INTO customeraccount (account_id, loyalty_score, reach_target, customer_id, level_id)
VALUES %s
"""

values = [
    (a["account_id"], a["loyalty_score"], a["reach_target"], a["customer_id"], a["level_id"])
    for a in customer_accounts
]

execute_values(cur, query, values, page_size=len(values))

conn.commit()